In [2]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image

In [4]:
pip install kagglehub[pandas-datasets]

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.0 MB 2.0 MB/s eta 0:00:05
   ------ --------------------------------- 1.8/11.0 MB 2.6 MB/s eta 0:00:04
   --------- ------------------------------ 2.6/11.0 MB 3.1 MB/s eta 0:00:03
   ------------- -------------------------- 3.7/11.0 MB 3.5 MB/s eta 0:00:03
   ----------------- ---------------------- 4.7/11.0 MB 3.8 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/11.0 MB 3.9 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.0 MB 4.0 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/

In [ ]:
import kagglehub

path = kagglehub.dataset_download("humansintheloop/semantic-segmentation-of-aerial-imagery")
print("Path to dataset files:", path)

100%|██████████| 29.6M/29.6M [00:05<00:00, 5.30MB/s]

Extracting files...


Path to dataset files: C:\Users\micha\.cache\kagglehub\datasets\humansintheloop\semantic-segmentation-of-aerial-imagery\versions\1


In [ ]:

class SegmentationGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, mask_dir, batch_size, image_size):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_filenames = os.listdir(image_dir)
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, idx):
        batch_files = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        images, masks = [], []

        for filename in batch_files:
            img = Image.open(os.path.join(self.image_dir, filename)).resize(self.image_size)
            mask_name = filename.replace('.jpg', '_mask.png')
            mask = Image.open(os.path.join(self.mask_dir, mask_name)).resize(self.image_size)

            images.append(np.array(img) / 255.0)
            masks.append(np.array(mask))

        return np.array(images), np.array(masks)

# Instantiate generator
images_path = r"C:\Users\micha\.cache\kagglehub\datasets\bulentsiyah\semantic-drone-dataset\versions\6\dataset\semantic_drone_dataset\original_images"
mask_path   = r"C:\Users\micha\.cache\kagglehub\datasets\bulentsiyah\semantic-drone-dataset\versions\6\RGB_color_image_masks\RGB_color_image_masks"

train_gen = SegmentationGenerator(images_path, mask_path, batch_size=4, image_size=(256, 256))
